# ICS5110: Applied Machine Learning

Peter Galea St. John: 288098m\
Christian A. Vella


In [1]:
# Import some necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import csv
import graphviz
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

######-ANN
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler


######-RF
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

# Data Preparation

In [2]:
 with open('Cleaned-Data.csv') as csv_file:
    data = list(csv.reader(csv_file, delimiter='\t'))
    r = len(data)

In [3]:
df = pd.read_csv('Cleaned-Data.csv',na_values="?" )

In [4]:
df.head()

,Fever,Tiredness,Dry-Cough,Difficulty-in-Breathing,Sore-Throat,None_Sympton,Pains,Nasal-Congestion,Runny-Nose,Diarrhea,...,Gender_Male,Gender_Transgender,Severity_Mild,Severity_Moderate,Severity_None,Severity_Severe,Contact_Dont-Know,Contact_No,Contact_Yes,Country
0,1,1,1,1,1,0,1,1,1,1,...,1,0,1,0,0,0,0,0,1,China
1,1,1,1,1,1,0,1,1,1,1,...,1,0,1,0,0,0,0,1,0,China
2,1,1,1,1,1,0,1,1,1,1,...,1,0,1,0,0,0,1,0,0,China
3,1,1,1,1,1,0,1,1,1,1,...,1,0,0,1,0,0,0,0,1,China
4,1,1,1,1,1,0,1,1,1,1,...,1,0,0,1,0,0,0,1,0,China


In [5]:
obj_df = df.select_dtypes(include=['object']).copy()
obj_df.head()

,Country
0,China
1,China
2,China
3,China
4,China


In [6]:
obj_df["Country"].value_counts()

Iran                  31680
Other                 31680
Spain                 31680
Republic of Korean    31680
China                 31680
France                31680
Germany               31680
UAE                   31680
Other-EUR             31680
Italy                 31680
Name: Country, dtype: int64

In [7]:
cleanup_Countries = {"Country": {"China":0, "UAE":1, "Iran":2, "France":3, "Republic of Korean":4, "Spain":5, "Italy":6, "Germany":7, "Other-EUR":8, "Other":9}}

In [8]:
obj_df = obj_df.replace(cleanup_Countries)

In [9]:
obj_df.head()

,Country
0,0
1,0
2,0
3,0
4,0


In [10]:
df["Country"] = obj_df

In [11]:
severity_columns = df.filter(like='Severity_').columns
gender_columns = df.filter(like='Gender_').columns

df['Severity_None'].replace({1:'0',0:'No'},inplace =True)
df['Severity_Mild'].replace({1:'1',0:'No'},inplace =True)
df['Severity_Moderate'].replace({1:'2',0:'No'},inplace =True)
df['Severity_Severe'].replace({1:'3',0:'No'},inplace =True)

df['Condition'] = df[severity_columns].values.tolist()

df.drop(columns=['Severity_None','Severity_Mild','Severity_Moderate','Severity_Severe'])

,Fever,Tiredness,Dry-Cough,Difficulty-in-Breathing,Sore-Throat,None_Sympton,Pains,Nasal-Congestion,Runny-Nose,Diarrhea,...,Age_25-59,Age_60+,Gender_Female,Gender_Male,Gender_Transgender,Contact_Dont-Know,Contact_No,Contact_Yes,Country,Condition
0,1,1,1,1,1,0,1,1,1,1,...,0,0,0,1,0,0,0,1,0,"[1, No, No, No]"
1,1,1,1,1,1,0,1,1,1,1,...,0,0,0,1,0,0,1,0,0,"[1, No, No, No]"
2,1,1,1,1,1,0,1,1,1,1,...,0,0,0,1,0,1,0,0,0,"[1, No, No, No]"
3,1,1,1,1,1,0,1,1,1,1,...,0,0,0,1,0,0,0,1,0,"[No, 2, No, No]"
4,1,1,1,1,1,0,1,1,1,1,...,0,0,0,1,0,0,1,0,0,"[No, 2, No, No]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316795,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,1,0,1,0,9,"[No, No, No, 3]"
316796,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,1,1,0,0,9,"[No, No, No, 3]"
316797,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,1,0,0,1,9,"[No, No, 0, No]"
316798,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,1,0,1,0,9,"[No, No, 0, No]"


In [12]:
def removing(list1):
    list1 = set(list1) 
    list1.discard("No")
    a = ''.join(list1)
    return a

In [13]:
df['Condition'] = df['Condition'].apply(removing)

In [14]:
df.head()

,Fever,Tiredness,Dry-Cough,Difficulty-in-Breathing,Sore-Throat,None_Sympton,Pains,Nasal-Congestion,Runny-Nose,Diarrhea,...,Gender_Transgender,Severity_Mild,Severity_Moderate,Severity_None,Severity_Severe,Contact_Dont-Know,Contact_No,Contact_Yes,Country,Condition
0,1,1,1,1,1,0,1,1,1,1,...,0,1,No,No,No,0,0,1,0,1
1,1,1,1,1,1,0,1,1,1,1,...,0,1,No,No,No,0,1,0,0,1
2,1,1,1,1,1,0,1,1,1,1,...,0,1,No,No,No,1,0,0,0,1
3,1,1,1,1,1,0,1,1,1,1,...,0,No,2,No,No,0,0,1,0,2
4,1,1,1,1,1,0,1,1,1,1,...,0,No,2,No,No,0,1,0,0,2


In [15]:
headers= df.columns.tolist()

In [16]:
df = df.sample(n=30000, replace=True,random_state=1)

In [17]:
X= pd.DataFrame.to_numpy(df.drop(['Condition'],axis=1))
y= pd.DataFrame.to_numpy(df['Condition'])

In [18]:
X= np.delete(X,19,1)
X= np.delete(X,19,1)
X= np.delete(X,19,1)
X= np.delete(X,19,1)

In [19]:
print(headers)
headers.remove('Severity_Mild')
headers.remove('Severity_Moderate')
headers.remove('Severity_None')
headers.remove('Severity_Severe')
headers.remove('Condition')
print(headers)

['Fever', 'Tiredness', 'Dry-Cough', 'Difficulty-in-Breathing', 'Sore-Throat', 'None_Sympton', 'Pains', 'Nasal-Congestion', 'Runny-Nose', 'Diarrhea', 'None_Experiencing', 'Age_0-9', 'Age_10-19', 'Age_20-24', 'Age_25-59', 'Age_60+', 'Gender_Female', 'Gender_Male', 'Gender_Transgender', 'Severity_Mild', 'Severity_Moderate', 'Severity_None', 'Severity_Severe', 'Contact_Dont-Know', 'Contact_No', 'Contact_Yes', 'Country', 'Condition']
['Fever', 'Tiredness', 'Dry-Cough', 'Difficulty-in-Breathing', 'Sore-Throat', 'None_Sympton', 'Pains', 'Nasal-Congestion', 'Runny-Nose', 'Diarrhea', 'None_Experiencing', 'Age_0-9', 'Age_10-19', 'Age_20-24', 'Age_25-59', 'Age_60+', 'Gender_Female', 'Gender_Male', 'Gender_Transgender', 'Contact_Dont-Know', 'Contact_No', 'Contact_Yes', 'Country']


In [20]:
y.shape

(30000,)

____

## Feature Selection: Chi Square

In [21]:
def get_max(x):
    max_val = 0
    for i in x:
        if i>max_val:
            max_val = i
    return max_val

In [22]:
def get_min(x):
    min_val = 99
    for i in x:
        if i<min_val:
            min_val = i
    return min_val

In [23]:
y = y.astype(np.int)

### Contingency Table

In [24]:
contingency_table = {headers[0]:{0,0}}

for x_col in range(X.shape[1]):
    col = X[:,x_col]
    h = headers[x_col]
    
    contingency_table[headers[x_col]] = {}
    
    for val_x in range(get_max(col)+1):
        count=0
        for val_y in range(get_max(y)+1):
            
            contingency_table[headers[x_col]][val_x,val_y]=  0

In [25]:
for x_col in range(X.shape[1]):
    col = X[:,x_col]
    print(headers[x_col])
    
    for i in range(col.shape[0]):
        contingency_table[headers[x_col]][col[i],y[i]] +=1

Fever
Tiredness
Dry-Cough
Difficulty-in-Breathing
Sore-Throat
None_Sympton
Pains
Nasal-Congestion
Runny-Nose
Diarrhea
None_Experiencing
Age_0-9
Age_10-19
Age_20-24
Age_25-59
Age_60+
Gender_Female
Gender_Male
Gender_Transgender
Contact_Dont-Know
Contact_No
Contact_Yes
Country


In [26]:
print(contingency_table[headers[0]][0,0])

5298


#### Degrees of Freedom

In [27]:
df ={}

for x_col in range(X.shape[1]):
    col = X[:,x_col]
    print(headers[x_col])
    
    rows = (get_max(col)+1)
    cols = (get_max(y)+1)
    
    df[headers[x_col]] = (rows-1)*(cols-1)
    

Fever
Tiredness
Dry-Cough
Difficulty-in-Breathing
Sore-Throat
None_Sympton
Pains
Nasal-Congestion
Runny-Nose
Diarrhea
None_Experiencing
Age_0-9
Age_10-19
Age_20-24
Age_25-59
Age_60+
Gender_Female
Gender_Male
Gender_Transgender
Contact_Dont-Know
Contact_No
Contact_Yes
Country


Degrees of Freedom for Contingency Tables = (r-1)*(c-1)

In [28]:
print(df)

{'Fever': 3, 'Tiredness': 3, 'Dry-Cough': 3, 'Difficulty-in-Breathing': 3, 'Sore-Throat': 3, 'None_Sympton': 3, 'Pains': 3, 'Nasal-Congestion': 3, 'Runny-Nose': 3, 'Diarrhea': 3, 'None_Experiencing': 3, 'Age_0-9': 3, 'Age_10-19': 3, 'Age_20-24': 3, 'Age_25-59': 3, 'Age_60+': 3, 'Gender_Female': 3, 'Gender_Male': 3, 'Gender_Transgender': 3, 'Contact_Dont-Know': 3, 'Contact_No': 3, 'Contact_Yes': 3, 'Country': 27}


### Expected Values

In [29]:
expected_values = {headers[0]:{0,0}}

for x_col in range(X.shape[1]):
    col = X[:,x_col]
    h = headers[x_col]
    
    expected_values[headers[x_col]] = {}
    Total = np.sum(list(contingency_table[headers[x_col]].values()))
    rowstotal = []
    colstotal = []
    
    for val_x in range(get_max(col)+1):
        
        for val_y in range(get_max(y)+1):
            rowstotal.append(contingency_table[headers[x_col]][val_x,val_y]) 
            
    for val_y in range(get_max(y)+1):
        
        for val_x in range(get_max(col)+1):
            colstotal.append(contingency_table[headers[x_col]][val_x,val_y])
    
    for val_x in range(get_max(col)+1):
        
        for val_y in range(get_max(y)+1):
            p = (rowstotal[val_x]/Total)*(colstotal[val_y]/Total)
            expected_values[headers[x_col]][val_x,val_y]=  p*Total
            

In [30]:
print(expected_values)

{'Fever': {(0, 0): 935.6268000000001, (0, 1): 415.5398, (0, 2): 895.1854, (0, 3): 419.7782, (1, 0): 895.1854, (1, 1): 397.57856666666663, (1, 2): 856.4920333333332, (1, 3): 401.63376666666665}, 'Tiredness': {(0, 0): 507.00000000000006, (0, 1): 487.63, (0, 2): 483.47, (0, 3): 484.51, (1, 0): 483.47, (1, 1): 464.99896666666666, (1, 2): 461.03203333333335, (1, 3): 462.0237666666667}, 'Dry-Cough': {(0, 0): 377.44083333333333, (0, 1): 480.74633333333327, (0, 2): 364.093, (0, 3): 471.09999999999997, (1, 0): 364.093, (1, 1): 463.7452, (1, 2): 351.21720000000005, (1, 3): 454.44000000000005}, 'Difficulty-in-Breathing': {(0, 0): 468.50003333333336, (0, 1): 487.6199333333334, (0, 2): 463.3764, (0, 3): 467.1254, (1, 0): 463.3764, (1, 1): 482.2872, (1, 2): 458.3088, (1, 3): 462.01680000000005}, 'Sore-Throat': {(0, 0): 906.1932000000002, (0, 1): 423.55060000000003, (0, 2): 886.2062, (0, 3): 407.90860000000004, (1, 0): 886.2062, (1, 1): 414.2087666666667, (1, 2): 866.6600333333332, (1, 3): 398.911766

### Calculate Chi-Square Value

In [31]:
ChiSquare=[]

for x_col in range(X.shape[1]):
    col = X[:,x_col]
    O_E2_divE=[]
    for val_x in range(get_max(col)+1):
        for val_y in range(get_max(y)+1):
            Observed = (contingency_table[headers[x_col]][val_x,val_y])
            Expected = expected_values[headers[x_col]][val_x,val_y]
            O_E = Observed-Expected
            O_E2 = O_E * O_E
            O_E2_divE.append(O_E2/Expected) 
    ChiSquare.append(np.sum(O_E2_divE))

In [32]:
print(ChiSquare)

[168384.33475722797, 178679.82035330965, 224548.69581150825, 183796.39087166867, 169696.80302300825, 847476.2642236493, 158022.19408370458, 206245.6628140825, 205418.30394904967, 159763.70925434006, 576819.4347981558, 255405.17863815947, 254371.45952805958, 252550.4685363993, 256182.32597447364, 251557.21367774432, 164812.12625076916, 164507.8864260789, 163392.77755801764, 165183.30131751084, 163296.80585817815, 164275.56079003363, 1105106.235385769]


### Accept or Reject the Null Hypothesis

With 95% confidence, we will check if the calculated Chi-Square Values will fall in the acceptance or rejection region. If the variables fall in acceptance region (are less than the critical Chi-Square value), we will conclude that the two variables are independent and cannot be selected for training.

In [33]:
drop = []
for h in range(len(headers)):
    print("Testing if variable %s is independent of variable Condition."%headers[h])
    deg = df[headers[h]]
    print("- Degrees of Freedom is %d"%(deg))
    if deg == 3:
        crit_val = 7.82
    if deg == 27:
        crit_val = 40.11
    print("- Critical Value is %d"%(crit_val))
    
    if(ChiSquare[h]>crit_val):
        print("-- Variable %s is dependent on variable Condition."%headers[h])
    else:
        print("-- Variable %s is independent on variable Condition."%headers[h])
        print("DROPPING VARIABLE: %s"%headers[h])
        drop.append(h)
        
X = np.delete(X,drop,1)
    

Testing if variable Fever is independent of variable Condition.
- Degrees of Freedom is 3
- Critical Value is 7
-- Variable Fever is dependent on variable Condition.
Testing if variable Tiredness is independent of variable Condition.
- Degrees of Freedom is 3
- Critical Value is 7
-- Variable Tiredness is dependent on variable Condition.
Testing if variable Dry-Cough is independent of variable Condition.
- Degrees of Freedom is 3
- Critical Value is 7
-- Variable Dry-Cough is dependent on variable Condition.
Testing if variable Difficulty-in-Breathing is independent of variable Condition.
- Degrees of Freedom is 3
- Critical Value is 7
-- Variable Difficulty-in-Breathing is dependent on variable Condition.
Testing if variable Sore-Throat is independent of variable Condition.
- Degrees of Freedom is 3
- Critical Value is 7
-- Variable Sore-Throat is dependent on variable Condition.
Testing if variable None_Sympton is independent of variable Condition.
- Degrees of Freedom is 3
- Critica

---

## Shuffle and Split

In [34]:
def shuffle_and_split(x,y):
    global x_train
    global x_test
    global y_train
    global y_test
    
    N = x.shape[0]
    P = np.arange(0,N,1)
    np.random.shuffle(P)
    lim= int(0.8*N)
    
    x_train = np.copy(x[P[0:lim],:])
    y_train = np.copy(y[P[0:lim]])
    x_test = np.copy(x[P[lim:N],:])
    y_test = np.copy(y[P[lim:N]])

In [35]:
shuffle_and_split(X,y)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(24000, 23)
(24000,)
(6000, 23)
(6000,)


In [48]:
print((x_train[0][-1]))
x_train[0]

2


array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1,
       2], dtype=object)

---

# Decision Tree

In [51]:
Fever_values = []
Tiredness_values = []
Dry_Cough_values=[]
Difficulty_Breathing_values = []
Sore_Throat_values = []
No_Symptoms_values=[]
Pains_values = []
Nasal_Congestion_values = []
Runny_Nose_values=[]
Diarrhea_values = []
None_Experiencing_Values = []
Age_0_9_Values =[]
Age_10_19_Values =[]
Age_20_24_Values =[]
Age_25_59_Values = []
Age_60_pls_Values = []
Gender_Female_Values =[]
Gender_Male_Values =[]
Gender_Trans_Values =[]
Contact_dk_Values = []
Contact_no_Values =[]
Contact_yes_Values =[]
Country_Values = []

In [52]:
#
classes = [0,1,2,3]
# features = headers
# #

# classes2code = {}
# code2classes = {}
# i=0
# for v in classes:
#     print(v)
#     classes2code[v]=i
#     code2classes[i]=v
#     i+=1
# #
# print(classes2code)

In [53]:
#
# convert dataset to numpy arrays
n = len(headers)  # number of features
m = len(x_train)  # number of instances
X = np.zeros([m,n],dtype=float)
Y = np.zeros([m],dtype=float)
for i in range(m):
    X[i,0] = x_train[i][0]
    X[i,1] = x_train[i][1]
    X[i,2] = x_train[i][2]
    X[i,3] = x_train[i][3]
    X[i,4] = x_train[i][4]
    X[i,5] = x_train[i][5]
    X[i,6] = x_train[i][6]
    X[i,7] = x_train[i][7]
    X[i,8] = x_train[i][8]
    X[i,9] = x_train[i][9]
    X[i,10] = x_train[i][10]
    X[i,11] = x_train[i][11]
    X[i,12] = x_train[i][12]
    X[i,13] = x_train[i][13]
    X[i,14] = x_train[i][14]
    X[i,15] = x_train[i][15]
    X[i,16] = x_train[i][16]
    X[i,17] = x_train[i][17]
    X[i,18] = x_train[i][18]
    X[i,19] = x_train[i][19]
    X[i,20] = x_train[i][20]
    X[i,21] = x_train[i][21]
    X[i,22] = x_train[i][22]
    Y[i] = y_train[i]

In [55]:
print (X)
print (Y)

[[0. 0. 0. ... 0. 1. 2.]
 [0. 0. 0. ... 1. 0. 3.]
 [0. 0. 1. ... 0. 0. 8.]
 ...
 [0. 1. 1. ... 0. 0. 9.]
 [0. 0. 0. ... 0. 1. 6.]
 [1. 1. 1. ... 1. 0. 8.]]
[3. 2. 2. ... 0. 3. 3.]


In [56]:
X.shape

(24000, 23)

In [57]:
Y.reshape(1,24000).ravel()

array([3., 2., 2., ..., 0., 3., 3.])

In [58]:
# dt_model = DecisionTreeClassifier(criterion='entropy', )
# dt_model.fit(X,Y)

DecisionTreeClassifier(criterion='entropy')

In [64]:
param_DT = {'criterion':['entropy','gini'], 
            'splitter': ['best','random'],
            'max_depth': [10, 15,20,25,30],
           'max_features': ['auto','sqrt', 'log2'],
           'min_samples_split': [5,7,10,15,20],
            'min_samples_leaf':[5,7,10,15,20],
            'max_leaf_nodes': [5,7,10,15,20],
            'min_impurity_decrease': [5,7,10,15,20],
            'class_weight': ['balanced']
            
           }
DT = DecisionTreeClassifier()
clf_2 = GridSearchCV(DT, param_DT, n_jobs=-1 ,cv=5, verbose = 100)

In [65]:
%time clf_2.fit(X,Y) 

Fitting 5 folds for each of 37500 candidates, totalling 187500 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elaps

[Parallel(n_jobs=-1)]: Done 156 tasks      | elapsed:   10.1s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:   10.2s
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed:   10.3s
[Parallel(n_jobs=-1)]: Done 162 tasks      | elapsed:   10.3s
[Parallel(n_jobs=-1)]: Done 164 tasks      | elapsed:   10.3s
[Parallel(n_jobs=-1)]: Done 166 tasks      | elapsed:   10.4s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   10.4s
[Parallel(n_jobs=-1)]: Done 170 tasks      | elapsed:   10.5s
[Parallel(n_jobs=-1)]: Done 172 tasks      | elapsed:   10.5s
[Parallel(n_jobs=-1)]: Done 174 tasks      | elapsed:   10.6s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   10.6s
[Parallel(n_jobs=-1)]: Done 178 tasks      | elapsed:   10.6s
[Parallel(n_jobs=-1)]: Done 180 tasks      | elapsed:   10.7s
[Parallel(n_jobs=-1)]: Done 182 tasks      | elapsed:   10.8s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   10.8s
[Parallel(n_jobs=-1)]: Done 186 tasks      | elapsed:   10.8s
[Paralle

[Parallel(n_jobs=-1)]: Done 422 tasks      | elapsed:   16.4s
[Parallel(n_jobs=-1)]: Done 424 tasks      | elapsed:   16.4s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:   16.5s
[Parallel(n_jobs=-1)]: Done 428 tasks      | elapsed:   16.5s
[Parallel(n_jobs=-1)]: Done 430 tasks      | elapsed:   16.5s
[Parallel(n_jobs=-1)]: Done 432 tasks      | elapsed:   16.6s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   16.7s
[Parallel(n_jobs=-1)]: Done 436 tasks      | elapsed:   16.7s
[Parallel(n_jobs=-1)]: Done 438 tasks      | elapsed:   16.7s
[Parallel(n_jobs=-1)]: Done 440 tasks      | elapsed:   16.8s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   16.8s
[Parallel(n_jobs=-1)]: Done 444 tasks      | elapsed:   16.9s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:   16.9s
[Parallel(n_jobs=-1)]: Done 448 tasks      | elapsed:   16.9s
[Parallel(n_jobs=-1)]: Done 450 tasks      | elapsed:   17.0s
[Parallel(n_jobs=-1)]: Done 452 tasks      | elapsed:   17.1s
[Paralle

[Parallel(n_jobs=-1)]: Done 694 tasks      | elapsed:   23.1s
[Parallel(n_jobs=-1)]: Done 696 tasks      | elapsed:   23.1s
[Parallel(n_jobs=-1)]: Done 698 tasks      | elapsed:   23.2s
[Parallel(n_jobs=-1)]: Done 700 tasks      | elapsed:   23.2s
[Parallel(n_jobs=-1)]: Done 702 tasks      | elapsed:   23.3s
[Parallel(n_jobs=-1)]: Done 704 tasks      | elapsed:   23.3s
[Parallel(n_jobs=-1)]: Done 706 tasks      | elapsed:   23.3s
[Parallel(n_jobs=-1)]: Done 708 tasks      | elapsed:   23.4s
[Parallel(n_jobs=-1)]: Done 710 tasks      | elapsed:   23.4s
[Parallel(n_jobs=-1)]: Done 712 tasks      | elapsed:   23.5s
[Parallel(n_jobs=-1)]: Done 714 tasks      | elapsed:   23.5s
[Parallel(n_jobs=-1)]: Done 716 tasks      | elapsed:   23.6s
[Parallel(n_jobs=-1)]: Done 718 tasks      | elapsed:   23.6s
[Parallel(n_jobs=-1)]: Done 720 tasks      | elapsed:   23.7s
[Parallel(n_jobs=-1)]: Done 722 tasks      | elapsed:   23.7s
[Parallel(n_jobs=-1)]: Done 724 tasks      | elapsed:   23.7s
[Paralle

[Parallel(n_jobs=-1)]: Done 968 tasks      | elapsed:   29.8s
[Parallel(n_jobs=-1)]: Done 970 tasks      | elapsed:   29.8s
[Parallel(n_jobs=-1)]: Done 972 tasks      | elapsed:   29.9s
[Parallel(n_jobs=-1)]: Done 974 tasks      | elapsed:   29.9s
[Parallel(n_jobs=-1)]: Done 976 tasks      | elapsed:   30.0s
[Parallel(n_jobs=-1)]: Done 978 tasks      | elapsed:   30.0s
[Parallel(n_jobs=-1)]: Done 980 tasks      | elapsed:   30.1s
[Parallel(n_jobs=-1)]: Done 982 tasks      | elapsed:   30.1s
[Parallel(n_jobs=-1)]: Done 984 tasks      | elapsed:   30.2s
[Parallel(n_jobs=-1)]: Done 986 tasks      | elapsed:   30.2s
[Parallel(n_jobs=-1)]: Done 988 tasks      | elapsed:   30.2s
[Parallel(n_jobs=-1)]: Done 990 tasks      | elapsed:   30.3s
[Parallel(n_jobs=-1)]: Done 992 tasks      | elapsed:   30.3s
[Parallel(n_jobs=-1)]: Done 994 tasks      | elapsed:   30.3s
[Parallel(n_jobs=-1)]: Done 996 tasks      | elapsed:   30.4s
[Parallel(n_jobs=-1)]: Done 998 tasks      | elapsed:   30.5s
[Paralle

...

[Parallel(n_jobs=-1)]: Done 185700 tasks      | elapsed: 43.0min
[Parallel(n_jobs=-1)]: Done 185704 tasks      | elapsed: 43.0min
[Parallel(n_jobs=-1)]: Done 185708 tasks      | elapsed: 43.0min
[Parallel(n_jobs=-1)]: Done 185712 tasks      | elapsed: 43.0min
[Parallel(n_jobs=-1)]: Done 185716 tasks      | elapsed: 43.0min
[Parallel(n_jobs=-1)]: Done 185720 tasks      | elapsed: 43.0min
[Parallel(n_jobs=-1)]: Done 185724 tasks      | elapsed: 43.0min
[Parallel(n_jobs=-1)]: Done 185728 tasks      | elapsed: 43.0min
[Parallel(n_jobs=-1)]: Done 185732 tasks      | elapsed: 43.0min
[Parallel(n_jobs=-1)]: Done 185736 tasks      | elapsed: 43.0min
[Parallel(n_jobs=-1)]: Done 185740 tasks      | elapsed: 43.0min
[Parallel(n_jobs=-1)]: Done 185744 tasks      | elapsed: 43.0min
[Parallel(n_jobs=-1)]: Done 185748 tasks      | elapsed: 43.0min
[Parallel(n_jobs=-1)]: Done 185752 tasks      | elapsed: 43.0min
[Parallel(n_jobs=-1)]: Done 185756 tasks      | elapsed: 43.0min
[Parallel(n_jobs=-1)]: Do

[Parallel(n_jobs=-1)]: Done 186224 tasks      | elapsed: 43.1min
[Parallel(n_jobs=-1)]: Done 186228 tasks      | elapsed: 43.1min
[Parallel(n_jobs=-1)]: Done 186232 tasks      | elapsed: 43.1min
[Parallel(n_jobs=-1)]: Done 186236 tasks      | elapsed: 43.1min
[Parallel(n_jobs=-1)]: Done 186240 tasks      | elapsed: 43.1min
[Parallel(n_jobs=-1)]: Done 186244 tasks      | elapsed: 43.1min
[Parallel(n_jobs=-1)]: Done 186248 tasks      | elapsed: 43.1min
[Parallel(n_jobs=-1)]: Done 186252 tasks      | elapsed: 43.1min
[Parallel(n_jobs=-1)]: Done 186256 tasks      | elapsed: 43.1min
[Parallel(n_jobs=-1)]: Done 186260 tasks      | elapsed: 43.1min
[Parallel(n_jobs=-1)]: Done 186264 tasks      | elapsed: 43.1min
[Parallel(n_jobs=-1)]: Done 186268 tasks      | elapsed: 43.1min
[Parallel(n_jobs=-1)]: Done 186272 tasks      | elapsed: 43.1min
[Parallel(n_jobs=-1)]: Done 186276 tasks      | elapsed: 43.1min
[Parallel(n_jobs=-1)]: Done 186280 tasks      | elapsed: 43.1min
[Parallel(n_jobs=-1)]: Do

[Parallel(n_jobs=-1)]: Done 186736 tasks      | elapsed: 43.2min
[Parallel(n_jobs=-1)]: Done 186740 tasks      | elapsed: 43.2min
[Parallel(n_jobs=-1)]: Done 186744 tasks      | elapsed: 43.2min
[Parallel(n_jobs=-1)]: Done 186748 tasks      | elapsed: 43.2min
[Parallel(n_jobs=-1)]: Done 186752 tasks      | elapsed: 43.2min
[Parallel(n_jobs=-1)]: Done 186756 tasks      | elapsed: 43.2min
[Parallel(n_jobs=-1)]: Done 186760 tasks      | elapsed: 43.2min
[Parallel(n_jobs=-1)]: Done 186764 tasks      | elapsed: 43.2min
[Parallel(n_jobs=-1)]: Done 186768 tasks      | elapsed: 43.2min
[Parallel(n_jobs=-1)]: Done 186772 tasks      | elapsed: 43.2min
[Parallel(n_jobs=-1)]: Done 186776 tasks      | elapsed: 43.2min
[Parallel(n_jobs=-1)]: Done 186780 tasks      | elapsed: 43.2min
[Parallel(n_jobs=-1)]: Done 186784 tasks      | elapsed: 43.2min
[Parallel(n_jobs=-1)]: Done 186788 tasks      | elapsed: 43.2min
[Parallel(n_jobs=-1)]: Done 186792 tasks      | elapsed: 43.2min
[Parallel(n_jobs=-1)]: Do

[Parallel(n_jobs=-1)]: Done 187252 tasks      | elapsed: 43.3min
[Parallel(n_jobs=-1)]: Done 187256 tasks      | elapsed: 43.3min
[Parallel(n_jobs=-1)]: Done 187260 tasks      | elapsed: 43.3min
[Parallel(n_jobs=-1)]: Done 187264 tasks      | elapsed: 43.3min
[Parallel(n_jobs=-1)]: Done 187268 tasks      | elapsed: 43.3min
[Parallel(n_jobs=-1)]: Done 187272 tasks      | elapsed: 43.3min
[Parallel(n_jobs=-1)]: Done 187276 tasks      | elapsed: 43.4min
[Parallel(n_jobs=-1)]: Done 187280 tasks      | elapsed: 43.4min
[Parallel(n_jobs=-1)]: Done 187284 tasks      | elapsed: 43.4min
[Parallel(n_jobs=-1)]: Done 187288 tasks      | elapsed: 43.4min
[Parallel(n_jobs=-1)]: Done 187292 tasks      | elapsed: 43.4min
[Parallel(n_jobs=-1)]: Done 187296 tasks      | elapsed: 43.4min
[Parallel(n_jobs=-1)]: Done 187300 tasks      | elapsed: 43.4min
[Parallel(n_jobs=-1)]: Done 187304 tasks      | elapsed: 43.4min
[Parallel(n_jobs=-1)]: Done 187308 tasks      | elapsed: 43.4min
[Parallel(n_jobs=-1)]: Do

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'class_weight': ['balanced'],
                         'criterion': ['entropy', 'gini'],
                         'max_depth': [10, 15, 20, 25, 30],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'max_leaf_nodes': [5, 7, 10, 15, 20],
                         'min_impurity_decrease': [5, 7, 10, 15, 20],
                         'min_samples_leaf': [5, 7, 10, 15, 20],
                         'min_samples_split': [5, 7, 10, 15, 20],
                         'splitter': ['best', 'random']},
             verbose=100)

In [61]:
print(clf_2.best_params_) 

{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 20, 'max_features': 'auto', 'max_leaf_nodes': 10, 'min_impurity_decrease': 10, 'min_samples_leaf': 10, 'min_samples_split': 10, 'splitter': 'best'}


In [63]:
y_pred_dt = clf_2.predict(x_test)
acc_dt =accuracy_score(y_test, y_pred_dt)
acc_dt 

0.24433333333333335

In [45]:
# predict and compare (you should check consistency)
print('y=',dt_model.predict(x_test))
print('Y=',Y)

y= [3. 0. 2. ... 1. 2. 0.]
Y= [0. 1. 3. ... 2. 3. 3.]


In [45]:
# predict and compare (you should check consistency)
print('y=',dt_model.predict(x_test))
print('Y=',Y)

y= [3. 0. 2. ... 1. 2. 0.]
Y= [0. 1. 3. ... 2. 3. 3.]


In [45]:
# predict and compare (you should check consistency)
print('y=',dt_model.predict(x_test))
print('Y=',Y)

y= [3. 0. 2. ... 1. 2. 0.]
Y= [0. 1. 3. ... 2. 3. 3.]


In [47]:
dot_data = tree.export_graphviz(dt_model, out_file=None,
                                feature_names=headers, 
                                class_names = str(classes))
graph = graphviz.Source(dot_data) 
%time graph.render("Assignment", view=True) 

Wall time: 35min 51s


'Assignment.pdf'

____

# Neural Network

## Activation Function Definitions

In [ ]:
def sigmoid(x):
  
  #return 1/(1 + np.exp(-x))
  return np.tanh(x)

def sigmoid_prime(x):
  
  #return sigmoid(x)*(1 - sigmoid(x))
  return 1 - np.power(np.tanh(x), 2)

## Weights for NN

In [ ]:
v = (2*np.random.random((2,3))-1)
w = (2*np.random.random((3,1))-1)

## Define the learning rate and the network structure

In [ ]:
alpha = 0.1

number_input_units = 3
number_hidden_units = 2 # ignore bias term
number_output_units = 1

## Perform the feedforward and backpropagation to update the weights and train the network

In [ ]:
error = []

for epoch in range(0, 2000):
    
  # randomly select a training example
  entry = np.random.randint(4)
    
  x = X[entry,:]

  # 1. Feedforward

  Z_in = []
  Z_out = []
    
  # Each hidden unit sums its weighted input signals
  for j in range(0, number_hidden_units):
    Z_in.append(np.dot(x, v[j,:]))
    Z_out.append(sigmoid(Z_in[j])) # and applies its activation function to computer its output
    
  
  # We append the bias term to layer_1
  Z_out = np.append(1, Z_out)
    
  Y_in = []
  Y_out = []
    
  # Each output unit sums its weighted input signals
  for k in range(0, number_output_units):
    Y_in.append(np.dot(Z_out, w))
    Y_out.append(sigmoid(Y_in[k]))
      
  Y_in = Y_in[0]
  Y_out = Y_out[0]
  
   
  # 2. Backpropagation

  delta_Y = (y[entry] - Y_out)*sigmoid_prime(Y_in)
    
  delta_Y = np.array(delta_Y)
   
  #err = y[entry]-Y_out
  err = np.power(y[entry]-Y_out, 2)
            
  error.append(err)
        
  delta_Z_in = []
  delta_Z = []
    
  # Each hidden unit sums its delta inputs from previous layer
  for j in range(0, number_hidden_units):
    delta_Z_in.append(delta_Y*w[j+1])
    delta_Z.append(delta_Z_in[j]*sigmoid_prime(Z_in[j]))
      
      
  delta_Z = np.array(delta_Z)
    
        
  # 3. Weights adjustment
           
  tmp = alpha*delta_Y*Z_out
     
  v = v + alpha*delta_Z*x
  w = np.add(w.T, tmp)
    
  w = w.T
         
    
fig = plt.figure()
plt.plot(error)
plt.ylabel('Error')
plt.xlabel('Iterations')
plt.show()